<a href="https://colab.research.google.com/github/cheng1103/Test/blob/main/104_Customization_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#104自訂條件職缺爬蟲(記得要填帳號密碼啊！！！)

In [ ]:
ID = "XXXXXXX" #輸入帳號
password = "XXXXXX" #輸入密碼
file_name = 'XXXXXX' #檔案命名
n = '1' #輸入自訂工作條件清單編號

job_op = '#li_newjob' + n

#暫停狀態
def lazy_time_s():
  t = random.uniform(2, 4) #隨機數
  time.sleep(t) #暫停

def lazy_time_l():
  t = random.uniform(3, 5) #隨機數
  time.sleep(t) #暫停

#python中grequests模塊簡單應用
#https://blog.csdn.net/cong_da_da/article/details/84325849

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
#!apt-get install xvfb
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.support.ui import WebDriverWait
#import selenium.webdriver.support.ui as ui

#!pip install grequests
#import grequests

!pip install fake_useragent
from fake_useragent import UserAgent


import requests
from bs4 import BeautifulSoup
from lxml import etree
import json
import os
import numpy as np
import pandas as pd
import random
import time

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#useragent = UserAgent().random #隨機請求表頭

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
#chrome_options.add_argument('--user-agent=%s' % useragent)
chrome_options.add_argument("--disable-blink-features")
#chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

wait = WebDriverWait(browser,10)
#wait.until(lambda driver: browser.find_element_by_css_selector('#bubjobs_li > a'))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
url = "https://www.104.com.tw/jobs/main/"
browser.get(url)
lazy_time_s() #等待

#進入登入頁面
signin = browser.find_element_by_css_selector('#global_bk > ul > li.right > ul > li:nth-child(6) > a')
browser.execute_script("arguments[0].click();", signin)
lazy_time_s() #

#填帳號密碼與登入
context = browser.find_element(by=By.XPATH, value='//*[@id="username"]')
context.send_keys(ID)
time.sleep(0.5) #暫停
context = browser.find_element(by=By.XPATH, value='//*[@id="password"]')
context.send_keys(password)
time.sleep(0.5) #暫停

submit = browser.find_element(by=By.XPATH, value='//*[@id="submitBtn"]/h2')
submit.click() #點擊登入
lazy_time_s() #等待



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  


In [ ]:
soup = BeautifulSoup(browser.page_source, 'html.parser') #解析網頁
print(soup)

In [ ]:
#顯示自訂工作列表
newjob = browser.find_element_by_css_selector('.newjob')
browser.execute_script("arguments[0].click();", newjob) #點擊
lazy_time_s() #等待

#自訂工作
newjob = browser.find_element_by_css_selector(job_op)
browser.execute_script("arguments[0].click();", newjob)

#檢查並切換頁面
n = browser.window_handles  # 生成一個新視窗或新頁籤的識別碼，代表這個視窗的模擬driver
print('當前識別碼:', n)  # 列印所有頁面識別碼
browser.switch_to.window(browser.window_handles[-1])  # 瀏覽器切换至最新跳出的頁面
lazy_time_l() #等待


#擷取總頁數
browser.execute_script("window.scrollTo(0,document.body.scrollHeight)") #捲動頁面至底端
time.sleep(10) #暫停
soup = BeautifulSoup(browser.page_source, 'html.parser') #解析網頁
lazy_time_l() #等待
last_page = soup.find('div', class_="pagination__count d-inline-block").getText()
last_page = last_page.split(' ')[-3]
last_page = int(last_page) + 1

for i in range(last_page):
  browser.execute_script("window.scrollTo(0,document.body.scrollHeight)") #捲動頁面至底端
  lazy_time_s() #等待
time.sleep(3) #暫停

#擷取職缺網址
soup = BeautifulSoup(browser.page_source, 'html.parser') #解析網頁
job_links = []
urls = soup.find_all('a', class_="info-job__text jb-link jb-link-blue jb-link-blue--visited h2")
for url in urls:
  link = url.get("href").replace('//', '').replace('?jobsource=m104_apply','')
  job_links.append(link)

browser.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  import sys


當前識別碼: ['CDwindow-217EA392E674B45F7BD2F81157C17FA5', 'CDwindow-E9F90D0A6AF78232C9347DDD5A746DCA']


In [ ]:
job_links

['www.104.com.tw/job/7ihkj',
 'www.104.com.tw/job/6th3c',
 'www.104.com.tw/job/75td6',
 'www.104.com.tw/job/788da',
 'www.104.com.tw/job/7awaw',
 'www.104.com.tw/job/7f69m',
 'www.104.com.tw/job/7kume',
 'www.104.com.tw/job/7ml62',
 'www.104.com.tw/job/5q64v',
 'www.104.com.tw/job/62cax',
 'www.104.com.tw/job/7aiqf',
 'www.104.com.tw/job/6l3ad',
 'www.104.com.tw/job/7ha1n',
 'www.104.com.tw/job/7j2f0',
 'www.104.com.tw/job/73o6h',
 'www.104.com.tw/job/75yy2',
 'www.104.com.tw/job/7mi4i',
 'www.104.com.tw/job/7mi8s',
 'www.104.com.tw/job/7hzns',
 'www.104.com.tw/job/7m5ro',
 'www.104.com.tw/job/78hpq',
 'www.104.com.tw/job/7dz8i',
 'www.104.com.tw/job/7jcsm',
 'www.104.com.tw/job/7jcs5',
 'www.104.com.tw/job/7jcrw',
 'www.104.com.tw/job/6xyim',
 'www.104.com.tw/job/7kmdu',
 'www.104.com.tw/job/79usr',
 'www.104.com.tw/job/7996y',
 'www.104.com.tw/job/7iwaa',
 'www.104.com.tw/job/7lr2z',
 'www.104.com.tw/job/7kwr6',
 'www.104.com.tw/job/7kwr8',
 'www.104.com.tw/job/7iv20',
 'www.104.com.

In [ ]:
data = pd.DataFrame()

for i in range(len(job_links)):

  #預處理網址
  link = job_links[i].replace('www.104.com.tw/job/','')
  url = 'https://www.104.com.tw/job/ajax/content/' + link
  Referer = 'https://www.104.com.tw/job/' + link


  #設定表頭
  useragent = UserAgent().random #隨機請求表頭
  headers = {'User-Agent':useragent, 'Referer':Referer }

  #發送請求
  req = requests.get(url, headers=headers)
  t = random.uniform(0.01, 0.5) #隨機數
  time.sleep(t) #暫停
  req = req.json()['data']

  #擷取資料
  date = req['header']['appearDate'] #刊登日期
  company = req['header']['custName'] #公司
  industry = req['industry'] #產業別
  jobName = req['header']['jobName'] #職銜
  jobDescription = req['jobDetail']['jobDescription']#.replace('\n', '　') #工作內容
  if len(jobDescription) <= 2 :
    jobDescription = '未填寫'

  jobcondition = req['condition']['other']#.replace('\n', '　') #工作條件-------------
  if len(jobcondition) <= 2 :
    jobcondition = '未填寫'
  
  businessTrip = req['jobDetail']['businessTrip'] #外派
  address = req['jobDetail']['addressRegion'] + req['jobDetail']['addressDetail'] + ' | ' + req['jobDetail']['landmark'] #地址
  exp = req['condition']['workExp'] #工作經驗

  expertise = req['condition']['specialty'] #專長-------------
  speciality = ''
  if len(expertise) == 0 :
    speciality = '不限'
  else:
    for i in range(len(expertise)):
      temp = expertise[i]['description']
      speciality = speciality + ' |' + temp

  certificate = req['condition']['certificate'] #證照-------------
  license = ''
  if len(certificate) == 0 :
    license = '未要求'
  else:
    for i in range(len(certificate)):
      temp = certificate[i]['name']
      license = license + ' |' + temp

  jobCategory = req['jobDetail']['jobCategory'] #職務類別-------------
  jobclass = ''
  for i in range(len(jobCategory)):
    temp = jobCategory[i]['description']
    jobclass = jobclass + ' |' + temp

  #彙整資料
  temp_data = {(date, company, industry, jobName, jobclass, jobDescription, jobcondition, speciality, exp, license, businessTrip, address, Referer)}
  temp_data = pd.DataFrame(temp_data)
  data = data.append(temp_data)

In [ ]:
  link = 'www.104.com.tw/job/7jzks'
  #預處理網址
  link = link.replace('www.104.com.tw/job/','')
  url = 'https://www.104.com.tw/job/ajax/content/' + link
  Referer = 'https://www.104.com.tw/job/' + link


  #設定表頭
  useragent = UserAgent().random #隨機請求表頭
  headers = {'User-Agent':useragent, 'Referer':Referer }

  #發送請求
  req = requests.get(url, headers=headers)
  t = random.uniform(0.1, 0.8) #隨機數
  time.sleep(t) #暫停
  req = req.json()['data']

  #擷取資料
  date = req['header']['appearDate'] #刊登日期
  company = req['header']['custName'] #公司
  industry = req['industry']
  jobName = req['header']['jobName'] #職銜
  jobDescription = req['jobDetail']['jobDescription']#.replace('\n', '　') #工作內容
  if len(jobDescription) == 0 :
    jobDescription = '未填寫'

  jobcondition = req['condition']['other']#.replace('\n', '　') #工作條件-------------
  if len(jobcondition) == 0 :
    jobcondition = '未填寫'
  
  businessTrip = req['jobDetail']['businessTrip'] #外派
  address = req['jobDetail']['addressRegion'] + req['jobDetail']['addressDetail'] + ' | ' + req['jobDetail']['landmark'] #地址
  exp = req['condition']['workExp'] #工作經驗

  expertise = req['condition']['specialty'] #專長-------------
  speciality = ''
  if len(expertise) == 0 :
    speciality = '不拘'
  else:
    for i in range(len(expertise)):
      temp = expertise[i]['description']
      speciality = speciality + ' |' + temp

  certificate = req['condition']['certificate'] #證照-------------
  license = ''
  if len(certificate) == 0 :
    license = '未要求'
  else:
    for i in range(len(certificate)):
      temp = certificate[i]['name']
      license = license + ' |' + temp

  jobCategory = req['jobDetail']['jobCategory'] #職務類別-------------
  jobclass = ''
  for i in range(len(jobCategory)):
    temp = jobCategory[i]['description']
    jobclass = jobclass + ' |' + temp

  #彙整資料
  temp_data = {(date, industry, company, jobName, jobclass, jobDescription, jobcondition, speciality, exp, license, businessTrip, address, Referer)}
  temp_data = pd.DataFrame(temp_data)
  


In [ ]:
data.columns = [ '日期', '公司', '產業別', '職銜', '職務類別', '工作內容', '其他條件', '擅長工具', '工作經驗', '證照', '出差與外派', '地區與地址', '網址']
data

,日期,公司,產業別,職銜,職務類別,工作內容,其他條件,擅長工具,工作經驗,證照,出差與外派,地區與地址,網址
0,2022/05/26,財團法人中衛發展中心,工商顧問服務業,G0-產業輔導暨技術發展顧問,|公共衛生人員 |顧問人員,產業輔導暨技術發展顧問:\n1. 協助衛福部護理機構評鑑輔導相關計畫執行。\n2. 協助部門...,1. 具醫療護理、公共衛生、統計等相關專業背景及工作經驗。\n2. 具備數據分析能力及使用分...,|Excel |PowerPoint |Word |Power BI​ |SPSS,2年以上,未要求,需出差，一年累積時間未定,台北市中正區杭州南路一段15-1號 | 距捷運善導寺站410公尺,https://www.104.com.tw/job/7ihkj
0,2022/05/26,財團法人中衛發展中心,工商顧問服務業,FF-數據分析顧問,|市場調查／市場分析 |顧問人員 |營運管理師／系統整合／ERP專案師,1. 執行產企業數據分析BI或AI輔導。\n2. 完成企業數據分析需求訪談，透過ETL建立數...,1. 工業工程、工業管理、資工或資管、機械工程、企管等相關科系碩士，或具備實務生產線經驗者。...,|Python |MS SQL,1年以上,未要求,需出差，一年累積時間未定,台北市中正區杭州南路一段15-1號3樓 (醒吾大樓) | 距捷運善導寺站410公尺,https://www.104.com.tw/job/6th3c
0,2022/05/26,財團法人中衛發展中心,工商顧問服務業,G3-農業經營顧問,|顧問人員 |其他專案管理師,1. 協助農業產業議題、產業分析、政策分析、計畫規劃統籌與規劃。\n2. 農業業者拜訪、資源...,1. 具農業、農經、商業、資訊等背景尤佳。\n2. 思緒清晰、謹慎細心，工作態度認真負責、具...,不限,2年以上,未要求,無需出差外派,台北市中正區杭州南路一段15-1號3樓 (醒吾大樓) | 距捷運善導寺站410公尺,https://www.104.com.tw/job/75td6
0,2022/05/26,財團法人中衛發展中心,工商顧問服務業,FF-程式系統工程師,|營運管理師／系統整合／ERP專案師 |軟體設計工程師 |系統分析師,Web平台系統開發、擴充:\n1. 完成與業者要求訪談，轉化為系統規格文件。\n2. 系統架...,外語能力依中心規定。,|ASP.NET |C# |Python,2年以上,未要求,無需出差外派,台北市中正區杭州南路一段15-1號3樓 (醒吾大樓) | 距捷運善導寺站410公尺,https://www.104.com.tw/job/788da
0,2022/05/26,財團法人中衛發展中心,工商顧問服務業,N4-產業研究員,|顧問人員 |其他專案管理師 |產品企劃開發人員,1.國內外服務業新興科技應用研究 (例如:VR、AR、5G、AI、IoT等)。\n2.輔導廠...,1. 大學以上學歷。\n2. 具有企業輔導經驗。\n3. 曾執行政府專案經驗。\n4. 外語...,|Excel |PowerPoint |Word,2年以上,未要求,需出差，一年累積時間未定,台北市中正區杭州南路一段15-1號3樓 (醒吾大樓) | 距捷運善導寺站410公尺,https://www.104.com.tw/job/7awaw
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2021/10/27,安泰商業銀行股份有限公司,銀行業,920 - 網站/行動服務企劃,|網站行銷企劃 |產品企劃開發人員 |廣告文案／企劃,協助銀行客群數位行銷企劃執行\r\n1.執行數位金融相關素材設計，包含活動網頁、各項Bann...,1.具宣傳素材設計完稿工作經驗(文宣、Banner、DM、EDM、活動頁、社群媒體(LINE...,不限,2年以上,未要求,無需出差外派,台北市信義區信義路五段7號41樓(台北101大樓) | 距捷運台北101/世貿站200公尺,https://www.104.com.tw/job/5pfab
0,2021/10/25,星展(台灣)商業銀行股份有限公司,銀行業,Contextual Marketing Platform Senior Data Analyst,|資料庫管理人員 |專案管理主管 |市場調查／市場分析,Job Overview: \nWe are looking for a CMP(Cont...,未填寫,不限,不拘,未要求,無需出差外派,台北市內湖區 |,https://www.104.com.tw/job/79nn5
0,2021/10/25,星展(台灣)商業銀行股份有限公司,銀行業,Software Engineer Team Lead,|軟體專案主管 |軟體設計工程師,1. Develop technology applications utilizing t...,1. Seasoned coders with people management expe...,不限,不拘,未要求,無需出差外派,台北市內湖區 |,https://www.104.com.tw/job/7b6iw
0,2021/10/25,安永聯合會計師事務所,會計服務業,【審計服務】財務會計諮詢服務 (Financial Accounting Advisory ...,|顧問人員 |查帳／審計人員,- 協助客戶進行財務流程優化提升營運效率\n- 輔導客戶會計準則轉換與導入相關財務新系統\n...,- 具會計 / 審計 / 內部稽核或系統導入專案兩年以上相關工作經驗\n- 對系統導入及流程...,|Excel |Outlook |PowerPoint |Word,2年以上,未要求,需出差，一年累積時間未定,台北市信義區基隆路一段333號9樓(110) | 距捷運台北101/世貿站220公尺,https://www.104.com.tw/job/7bbm1


In [ ]:
#輸出檔案
path = '/content/drive/MyDrive/Data Set/'+file_name + '.csv' #路徑
data.to_csv(path, index = None)